# LangGraph Translation Pipeline - Comprehensive Demo

This notebook demonstrates the **multi-stage translation pipeline** with all new features:

## Pipeline Stages:
1. **Sense Analysis** - Understand semantic nuances and context
2. **Definition Translation** - Translate definition with cultural adaptation
3. **Initial Translation** - Direct translation of each lemma
4. **Synonym Expansion** - Iteratively broaden candidate pool (NEW: up to 5 iterations)
5. **Synonym Filtering** - Quality check with per-word confidence (NEW: definition-anchored validation)
6. **Definition Quality Review** - Automated grammar/style check with optional revisions (NEW)
7. **Result Assembly** - Combine outputs into final synset

## New Features Demonstrated:
- ✨ **Iterative Expansion**: Runs expansion multiple times until convergence
- ✨ **Definition-Anchored Filtering**: Validates synonyms against translated definition
- ✨ **Definition Quality Review**: Post-filter safeguard that can revise definitions and log issues
- ✨ **Per-Word Confidence**: Individual quality scores for each synonym
- ✨ **Full Log Access**: Untruncated LLM outputs for analysis
- ✨ **Definition Comparison**: Compare translated definitions with existing Serbian glosses
- ✨ **Serbian WordNet Comparison**: Compare with existing human translations

## Why Definitions Matter:
📖 **Definitions are the foundation of WordNet structure**
- They define semantic boundaries of each synset
- They determine which synonyms belong together
- They enable precise cross-lingual alignment
- Matching definitions = same concept, even with different synonym choices

## 1️⃣ Setup

In [ ]:
# Import libraries
from pathlib import Path
import json
import importlib
from time import perf_counter
import ollama
import wordnet_autotranslate.pipelines.langgraph_translation_pipeline as lg_module

# Reload module to get latest changes
lg_module = importlib.reload(lg_module)
LangGraphTranslationPipeline = lg_module.LangGraphTranslationPipeline

print("✅ Imports complete")

✅ Imports complete


In [2]:
# Load data
DATA_PATH = Path("../examples/serbian_english_synset_pairs_enhanced.json")
with DATA_PATH.open("r", encoding="utf-8") as f:
    dataset = json.load(f)

pairs = dataset["pairs"]
print(f"✅ Loaded {len(pairs)} English-Serbian synset pairs")

✅ Loaded 27 English-Serbian synset pairs


In [3]:
# Configure Ollama
PREFERRED_MODEL = "gpt-oss:120b"
TIMEOUT = 180
TEMPERATURE = 0.0

# Check available models
model_list = ollama.list()
available = {m.model for m in model_list.models}

if PREFERRED_MODEL in available:
    model = PREFERRED_MODEL
else:
    model = sorted(available)[0]
    print(f"⚠️  Preferred model '{PREFERRED_MODEL}' not found, using '{model}'")

print(f"✅ Using model: {model}")

✅ Using model: gpt-oss:120b


In [4]:
# Initialize pipeline with iterative expansion
pipeline = LangGraphTranslationPipeline(
    source_lang="en",
    target_lang="sr",
    model=model,
    temperature=TEMPERATURE,
    timeout=TIMEOUT,
    max_expansion_iterations=5  # NEW: Iterative expansion
)

print("✅ Pipeline initialized")
print(f"   Max expansion iterations: {pipeline.max_expansion_iterations}")

✅ Pipeline initialized
   Max expansion iterations: 5


## 2️⃣ Detailed Translation Example

Let's translate the first synset and examine each stage in detail.

In [5]:
# Prepare first synset
pair_0 = pairs[0]
synset_0 = {
    "id": pair_0["english_id"],
    "lemmas": pair_0["english_lemmas"],
    "definition": pair_0["english_definition"],
    "examples": pair_0.get("english_examples", []),
    "pos": pair_0["english_pos"],
}

print("📋 Synset to translate:")
print(f"   ID: {synset_0['id']}")
print(f"   Lemmas: {', '.join(synset_0['lemmas'])}")
print(f"   Definition: {synset_0['definition']}")
print(f"   POS: {synset_0['pos']}")

📋 Synset to translate:
   ID: ENG30-03574555-n
   Lemmas: institution
   Definition: an establishment consisting of a building or complex of buildings where an organization for the promotion of some cause is situated
   POS: n


In [ ]:
# Run translation (this takes ~5-10 minutes with iterative expansion)
print("🔄 Running translation pipeline...\n")
single_start = perf_counter()
result_0 = pipeline.translate_synset(synset_0)
single_execution_seconds = perf_counter() - single_start
if "synset_execution_seconds" not in globals():
    synset_execution_seconds = {}
synset_execution_seconds[synset_0["id"]] = single_execution_seconds
print(f"✅ Translation complete in {single_execution_seconds:.2f} seconds!")

🔄 Running translation pipeline...

[Expansion] Iteration 1: Added 6 new synonym(s), total: 7
[Expansion] Iteration 2: Added 2 new synonym(s), total: 9
[Expansion] Iteration 3: Added 5 new synonym(s), total: 14
[Expansion] Iteration 4: Added 4 new synonym(s), total: 18
[Expansion] Iteration 5: Added 1 new synonym(s), total: 19
✅ Translation complete!


### Stage-by-Stage Breakdown

In [7]:
# Extract stage payloads
payload_0 = result_0.get("payload", {})
sense_0 = payload_0.get("sense", {})
definition_0 = payload_0.get("definition", {})
initial_0 = payload_0.get("initial_translation", {})
expansion_0 = payload_0.get("expansion", {})
filtering_0 = payload_0.get("filtering", {})
definition_quality_0 = payload_0.get("definition_quality", {})

curator_summary_0 = result_0.get("curator_summary", "")
final_definition_0 = result_0.get("definition_translation", "")
final_notes_0 = result_0.get("notes")

print("=" * 80)
print(f"SYNSET: {synset_0['id']}")
print(f"English Lemmas: {', '.join(synset_0['lemmas'])}")
print(f"POS: {synset_0['pos']}")
print("=" * 80)

print("\n" + "=" * 80)
print("STAGE 1: SENSE ANALYSIS")
print("=" * 80)
print(f"\nSense summary: {sense_0.get('sense_summary', '—')}")
print(f"Confidence: {sense_0.get('confidence', 'N/A')}")
print(f"Key features: {', '.join(sense_0.get('key_features', [])) or '—'}")

print("\n" + "=" * 80)
print("STAGE 2: DEFINITION TRANSLATION")
print("=" * 80)
print(f"\n🇬🇧 English definition:")
print(f"   {synset_0['definition']}")
print(f"\n🇷🇸 Serbian translation (pre-review):")
print(f"   {definition_0.get('definition_translation', '—')}")
print(f"\n📚 Existing Serbian gloss (from WordNet):")
print(f"   {pair_0['serbian_definition']}")

print("\n" + "=" * 80)
print("STAGE 3: INITIAL TRANSLATION")
print("=" * 80)
translations = initial_0.get('initial_translations', [])
print(f"\nTranslated {len(translations)} lemmas:")
for i, trans in enumerate(translations, 1):
    print(f"  {i}. {trans}")

print("\n" + "=" * 80)
print("STAGE 4: ITERATIVE EXPANSION")
print("=" * 80)
expanded = expansion_0.get('expanded_synonyms', [])
iterations = expansion_0.get('iterations_run', 1)
converged = expansion_0.get('converged', False)
print(f"\n🔄 Iterations: {iterations}")
print(f"✓ Converged: {'Yes' if converged else 'No (hit max limit)'}")
print(f"📊 Total synonyms: {len(expanded)}")
print(f"\nExpanded synonyms: {', '.join(expanded) if expanded else '—'}")

# Show synonym provenance
provenance = expansion_0.get('synonym_provenance', {})
if provenance:
    iter_counts = {}
    for syn, iter_num in provenance.items():
        iter_counts[iter_num] = iter_counts.get(iter_num, 0) + 1
    
    print(f"\n📈 Synonyms by iteration:")
    for iter_num in sorted(iter_counts.keys()):
        count = iter_counts[iter_num]
        if iter_num == 0:
            print(f"   Initial: {count} synonyms")
        else:
            print(f"   Iteration {iter_num}: {count} new synonyms")

print("\n" + "=" * 80)
print("STAGE 5: FILTERING")
print("=" * 80)
filtered = filtering_0.get('filtered_synonyms', [])
removed_items = filtering_0.get('removed', [])
confidence_by_word = filtering_0.get('confidence_by_word', {})
overall_conf = filtering_0.get('confidence', 'unknown')

print(f"\n✅ Kept: {len(filtered)} synonyms (overall confidence: {overall_conf})")
print(f"❌ Removed: {len(removed_items)} candidates")
print(f"\nFinal synonyms: {', '.join(filtered) if filtered else '—'}")

if confidence_by_word:
    print(f"\n🎯 Per-word confidence:")
    for word, conf in confidence_by_word.items():
        emoji = "🟢" if conf == "high" else "🟡" if conf == "medium" else "🔴"
        print(f"   {emoji} {word:20} → {conf}")

if removed_items:
    print(f"\n❌ Removed candidates:")
    for item in removed_items:
        word = item.get('word', '?')
        reason = item.get('reason', 'No reason')
        print(f"   • {word:20} → {reason}")

print("\n" + "=" * 80)
print("STAGE 6: DEFINITION QUALITY REVIEW")
print("=" * 80)
quality_status = definition_quality_0.get('status', 'unknown') or 'unknown'
print(f"\nStatus: {quality_status}")
revised_definition = definition_quality_0.get('revised_definition')
if revised_definition:
    print(f"\n📝 Revised definition:")
    print(f"   {revised_definition}")
else:
    print(f"\n📝 Revised definition: not required")

issues = definition_quality_0.get('issues') or []
if issues:
    print(f"\n⚠️ Issues flagged:")
    for issue in issues:
        issue_type = issue.get('type', 'unknown')
        message = issue.get('message', '')
        print(f"   - {issue_type}: {message}")

quality_notes = definition_quality_0.get('notes')
if quality_notes:
    print(f"\n🗒️ Review notes: {quality_notes}")

print("\n" + "=" * 80)
print("STAGE 7: RESULT ASSEMBLY")
print("=" * 80)
print(f"\nRepresentative literal: {result_0.get('translation', '—')}")
print(f"Definition used in final result: {final_definition_0 or '—'}")
if final_notes_0:
    print(f"Curator notes: {final_notes_0}")
if curator_summary_0:
    print(f"\n📋 Curator summary:\n{curator_summary_0}")

SYNSET: ENG30-03574555-n
English Lemmas: institution
POS: n

STAGE 1: SENSE ANALYSIS

Sense summary: A concrete establishment—a building or group of buildings—that houses an organization dedicated to advancing a particular cause or purpose.
Confidence: high
Key features: physical structure (building or complex), hosts an organization, purpose‑driven (promotion of a cause), formal, established location

STAGE 2: DEFINITION TRANSLATION

🇬🇧 English definition:
   an establishment consisting of a building or complex of buildings where an organization for the promotion of some cause is situated

🇷🇸 Serbian translation (pre-review):
   zgrada ili kompleks zgrada u kome se nalazi organizacija za promovisanje nekog cilja

📚 Existing Serbian gloss (from WordNet):
   zgrada u kojoj se nalazi organizaciona jedinica neke grane javnog poslovanja

STAGE 3: INITIAL TRANSLATION

Translated 1 lemmas:
  1. ustanova

STAGE 4: ITERATIVE EXPANSION

🔄 Iterations: 5
✓ Converged: No (hit max limit)
📊 Total sy

### 7️⃣ Definition Quality Verification
The seventh stage wires the grammar/style review back into the assembled result. The cells below surface the logged status, any issues, and the curator summary so you can confirm the definition-quality node executed as expected.

In [8]:
from pprint import pprint

definition_quality_log = payload_0.get("logs", {}).get("definition_quality", {})
definition_quality_call = payload_0.get("calls", {}).get("definition_quality", {})
definition_quality_payload = definition_quality_call.get("payload", {})

print("Definition quality log preview:")
pprint(definition_quality_log)

print("\nFull payload returned by the definition-quality node:")
pprint(definition_quality_payload)

summary_tail = curator_summary_0.splitlines()[-4:] if curator_summary_0 else []
print("\nCurator summary excerpt:")
print("\n".join(summary_tail) if summary_tail else "(no curator summary available)")

Definition quality log preview:
{'prompt': 'Evaluate the linguistic and stylistic quality of this Unknown (sr) '
           'dictionary definition.\n'
           '\n'
           'Definition:\n'
           '"zgrada ili kompleks zgrada u kome se nalazi organizacija za '
           'promovisanje nekog cilja"\n'
           '\n'
           'Synonyms (lemmas):\n'
           'administrativna zgrada, administrativni objekat, centar, centrala, '
           'filijala, glavna zgrada, radna zgrada, sedište\n'
           '\n'
           'Tasks:\n'
           '1. **Circularity** — Check if any lemma word or its close '
           'inflectional form\n'
           '   appears directly in the definition, which would create a '
           'circular definition.\n'
           '   If this occurs, suggest a neutral rephrasing that avoids '
           'repetition.\n'
           '\n'
           '2. **Grammar and agreement** — Verify correctness of inflection, '
           'case,\n'
           '   number, and 

## 3️⃣ Batch Translation

Now let's translate 4 more synsets to see how the pipeline handles different types of words.

In [9]:
# Prepare the full dataset for batch translation
batch_synsets: list[dict] = []
batch_pairs: list[dict] = []
all_display_names: list[str] = []

max_preview = 10
for idx, pair in enumerate(pairs):
    synset = {
        "id": pair["english_id"],
        "lemmas": pair["english_lemmas"],
        "definition": pair["english_definition"],
        "examples": pair.get("english_examples", []),
        "pos": pair["english_pos"],
    }

    batch_synsets.append(synset)
    batch_pairs.append(pair)

    display_name = pair.get("english_name") or ", ".join(synset["lemmas"][:2])
    if not display_name:
        display_name = synset["id"]
    all_display_names.append(display_name)

    if idx < max_preview:
        print(f"{idx + 1}. {synset['id']} ({synset['pos']})")
        print(f"   Lemmas: {', '.join(synset['lemmas'][:3])}")
        definition_snippet = synset['definition'][:80] + ("..." if len(synset['definition']) > 80 else "")
        print(f"   Definition: {definition_snippet}\n")

all_synsets = batch_synsets

print(f"✅ Prepared {len(batch_synsets)} synsets for translation")
if len(batch_synsets) > max_preview:
    print(f"   (Previewed the first {max_preview}; the dataset contains {len(batch_synsets)} total synsets.)")

1. ENG30-07810907-n (n)
   Lemmas: condiment
   Definition: a preparation (a sauce or relish or spice) to enhance flavor...

2. ENG30-01376245-v (v)
   Lemmas: scatter, sprinkle
   Definition: distribute loosely...

3. ENG30-01382083-v (v)
   Lemmas: pick, pluck
   Definition: look for and gather...

4. ENG30-01393996-v (v)
   Lemmas: sweep
   Definition: clean by sweeping...

✅ Prepared 4 synsets for translation


In [ ]:
# Translate the entire dataset (can take a while)
if "batch_synsets" not in globals() or not batch_synsets:
    raise RuntimeError("Run the preparation cell first to populate `batch_synsets`.")

print(f"🔄 Translating {len(batch_synsets)} synsets...\n")
batch_results: list[dict] = []

single_synset_id = synset_0["id"] if "synset_0" in globals() else None
single_result = result_0 if "result_0" in globals() else None

if "synset_execution_seconds" not in globals():
    synset_execution_seconds: dict[str, float] = {}

batch_execution_breakdown: list[dict] = []
batch_timer_start = perf_counter()

for idx, (synset, pair, name) in enumerate(zip(batch_synsets, batch_pairs, all_display_names), start=1):
    print(f"[{idx}/{len(batch_synsets)}] Translating {synset['id']} — {name}")

    if single_synset_id and synset["id"] == single_synset_id and single_result is not None:
        batch_results.append(single_result)
        reused_duration = synset_execution_seconds.get(synset["id"])
        if reused_duration is None:
            reused_duration = 0.0
            synset_execution_seconds[synset["id"]] = reused_duration
        batch_execution_breakdown.append({
            "synset_id": synset["id"],
            "display_name": name,
            "seconds": reused_duration,
            "reused_single_demo": True,
        })
        print(f"   ⏭️ Using existing single-synset result to avoid duplicate work (previous runtime: {reused_duration:.2f}s).\n")
        continue

    synset_timer_start = perf_counter()
    result = pipeline.translate_synset(synset)
    duration_seconds = perf_counter() - synset_timer_start
    synset_execution_seconds[synset["id"]] = duration_seconds
    batch_execution_breakdown.append({
        "synset_id": synset["id"],
        "display_name": name,
        "seconds": duration_seconds,
        "reused_single_demo": False,
    })
    batch_results.append(result)

    payload = result.get("payload", {}) if isinstance(result, dict) else {}
    filtering = payload.get("filtering", {})
    filtered = filtering.get("filtered_synonyms", []) or []
    conf = filtering.get("confidence", "unknown")
    print(f"   ✅ {len(filtered)} synonyms retained (confidence: {conf})")
    print(f"   ⏱️ Execution time: {duration_seconds:.2f} seconds\n")

batch_total_seconds = perf_counter() - batch_timer_start
sum_synset_seconds = sum(entry.get("seconds") or 0.0 for entry in batch_execution_breakdown)
batch_execution_summary = {
    "total_seconds": batch_total_seconds,
    "sum_synset_seconds": sum_synset_seconds,
    "per_synset": batch_execution_breakdown,
}

all_results = batch_results
print("✅ Batch translation complete!")
print(f"⏱️ Total elapsed time: {batch_total_seconds:.2f} seconds (sum of per-synset timers: {sum_synset_seconds:.2f} seconds)")

🔄 Translating 4 synsets...

[1/4] Translating ENG30-07810907-n...
[Expansion] Iteration 1: Added 7 new synonym(s), total: 8
[Expansion] Iteration 2: Added 1 new synonym(s), total: 9
[Expansion] Iteration 3: Added 4 new synonym(s), total: 13
[Expansion] Iteration 4: Added 1 new synonym(s), total: 14
[Expansion] Iteration 5: Added 2 new synonym(s), total: 16
   ✅ 8 synonyms, confidence: high

[2/4] Translating ENG30-01376245-v...
[Expansion] Iteration 1: Added 4 new synonym(s), total: 8
[Expansion] Iteration 2: Added 1 new synonym(s), total: 9
[Expansion] Converged after 3 iteration(s) - no new synonyms found
   ✅ 8 synonyms, confidence: high

[3/4] Translating ENG30-01382083-v...
[Expansion] Iteration 1: Added 5 new synonym(s), total: 6
[Expansion] Iteration 2: Added 3 new synonym(s), total: 9
[Expansion] Converged after 3 iteration(s) - no new synonyms found
   ✅ 7 synonyms, confidence: high

[4/4] Translating ENG30-01393996-v...
[Expansion] Iteration 1: Added 5 new synonym(s), total: 

### Standardized Analysis for Each Synset

In [ ]:
# Analyze all translated synsets in a standardized format
if "batch_results" not in globals() or not batch_results:
    raise RuntimeError("Run the batch translation cell before analyzing results.")

execution_summary = globals().get("batch_execution_summary")
synset_execution_seconds = globals().get("synset_execution_seconds", {})
if execution_summary:
    total_seconds = execution_summary.get("total_seconds")
    measured_sum = execution_summary.get("sum_synset_seconds")
    total_synsets = len(batch_results)
    if total_seconds is not None:
        print(f"⏱️ Batch runtime: {total_seconds:.2f} seconds")
    if measured_sum is not None and total_synsets:
        avg_seconds = measured_sum / total_synsets if total_synsets else 0.0
        print(f"   • Sum of per-synset timers: {measured_sum:.2f} seconds")
        print(f"   • Average per synset: {avg_seconds:.2f} seconds")

for idx, (name, synset, result, serbian_pair) in enumerate(
    zip(all_display_names, batch_synsets, batch_results, batch_pairs), start=1
):
    print("\n" + "=" * 80)
    print(f"SYNSET {idx}: {name.upper()}")
    print(f"ID: {synset['id']}")
    print(f"English Lemmas: {', '.join(synset['lemmas'])}")
    print(f"POS: {synset['pos']}")
    print("=" * 80)

    per_synset_time = synset_execution_seconds.get(synset['id'])
    if per_synset_time is not None:
        print(f"   ⏱️ Execution time: {per_synset_time:.2f} seconds")

    payload = result.get('payload', {}) if isinstance(result, dict) else {}
    definition_payload = payload.get('definition', {}) or {}
    definition_quality = payload.get('definition_quality', {}) or {}

    pre_review_definition = definition_payload.get('definition_translation', '')
    revised_definition = definition_quality.get('revised_definition')
    final_definition = result.get('definition_translation', pre_review_definition)

    print(f"\n📖 DEFINITIONS:")
    print(f"   🇬🇧 English: {synset['definition']}")
    print(f"   🇷🇸 Pipeline (final): {final_definition or '—'}")
    if revised_definition:
        print(f"   📝 Revised during quality review: {revised_definition}")
    if pre_review_definition and pre_review_definition != final_definition:
        print(f"   🔁 Pre-review version: {pre_review_definition}")
    print(f"   📚 Existing: {serbian_pair['serbian_definition']}")

    expansion = payload.get('expansion', {}) or {}
    filtering = payload.get('filtering', {}) or {}

    expanded = expansion.get('expanded_synonyms', []) or []
    filtered = filtering.get('filtered_synonyms', []) or []
    removed = filtering.get('removed', []) or []
    confidence_by_word = filtering.get('confidence_by_word', {}) or {}
    overall_conf = filtering.get('confidence', 'unknown')

    print(f"\n📊 Pipeline progression:")
    print(f"   Expanded: {len(expanded)} candidates")
    print(f"   Filtered: {len(filtered)} synonyms (confidence: {overall_conf})")
    print(f"   Removed: {len(removed)} items")

    iterations = expansion.get('iterations_run', 1)
    converged = expansion.get('converged', False)
    print(f"\n🔄 Expansion: {iterations} iteration(s), converged: {converged}")

    if confidence_by_word:
        print(f"\n🎯 Confidence distribution:")
        high = sum(1 for c in confidence_by_word.values() if c == "high")
        medium = sum(1 for c in confidence_by_word.values() if c == "medium")
        low = sum(1 for c in confidence_by_word.values() if c == "low")
        total_words = len(confidence_by_word)
        print(f"   🟢 High: {high}/{total_words} ({high/total_words*100:.0f}%)")
        print(f"   🟡 Medium: {medium}/{total_words} ({medium/total_words*100:.0f}%)")
        print(f"   🔴 Low: {low}/{total_words} ({low/total_words*100:.0f}%)")

    print(f"\n✨ Final synset: {', '.join(filtered) if filtered else '—'}")

    quality_status = definition_quality.get('status', 'unknown') or 'unknown'
    issues = definition_quality.get('issues') or []
    quality_notes = definition_quality.get('notes')

    print(f"\n🧹 Definition quality review: {quality_status}")
    if issues:
        print(f"   ⚠️ Issues: {len(issues)} flagged")
        for issue in issues:
            print(f"     - {issue.get('type', 'unknown')}: {issue.get('message', '')}")
    if quality_notes:
        print(f"   🗒️ Notes: {quality_notes}")


SYNSET 1: INSTITUTION
ID: ENG30-03574555-n
English Lemmas: institution
POS: n

📖 DEFINITIONS:
   🇬🇧 English: an establishment consisting of a building or complex of buildings where an organization for the promotion of some cause is situated
   🇷🇸 Pipeline (final): objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   📝 Revised during quality review: objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   🔁 Pre-review version: zgrada ili kompleks zgrada u kome se nalazi organizacija za promovisanje nekog cilja
   📚 Existing: zgrada u kojoj se nalazi organizaciona jedinica neke grane javnog poslovanja

📊 Pipeline progression:
   Expanded: 19 candidates
   Filtered: 8 synonyms (confidence: high)
   Removed: 11 items

🔄 Expansion: 5 iteration(s), converged: False

🎯 Confidence distribution:
   🟢 High: 6/8 (75%)
   🟡 Medium: 2/8 (25%)
   🔴 Low: 0/8 (0%)

✨ Final synset: administrativna zgrada, administrat

## 4️⃣ Comparative Analysis

In [ ]:
# Summary table
if "batch_results" not in globals() or not batch_results:
    raise RuntimeError("Run the batch translation cell before generating the summary table.")

execution_summary = globals().get("batch_execution_summary")
synset_execution_seconds = globals().get("synset_execution_seconds", {})

print("=" * 112)
print(f"COMPARATIVE SUMMARY: {len(batch_results)} Synsets")
print("=" * 112)

header = f"\n{'#':<3} {'Synset ID':<18} {'Lemmas':<25} {'POS':<5} {'Expanded':<10} {'Filtered':<10} {'Removed':<10} {'Confidence':<12} {'Def.Review':<12} {'Runtime(s)':<12}"
print(header)
print("-" * 124)

quality_counts = {"ok": 0, "needs_revision": 0, "unknown": 0}
revision_count = 0

for i, (name, synset, result) in enumerate(zip(all_display_names, batch_synsets, batch_results), 1):
    payload = result.get('payload', {}) if isinstance(result, dict) else {}
    expansion = payload.get('expansion', {}) or {}
    filtering = payload.get('filtering', {}) or {}
    definition_quality = payload.get('definition_quality', {}) or {}

    synset_id = synset['id']
    lemmas = ', '.join(synset['lemmas'][:2]) + ('...' if len(synset['lemmas']) > 2 else '')
    pos = synset['pos']
    expanded_count = len(expansion.get('expanded_synonyms', []) or [])
    filtered_count = len(filtering.get('filtered_synonyms', []) or [])
    removed_count = len(filtering.get('removed', []) or [])
    confidence = filtering.get('confidence', 'unknown')

    quality_status = (definition_quality.get('status') or 'unknown').lower()
    if quality_status not in quality_counts:
        quality_counts['unknown'] += 1
    else:
        quality_counts[quality_status] += 1

    revised = bool(definition_quality.get('revised_definition'))
    if revised:
        revision_count += 1

    def_review_display = quality_status
    if revised:
        def_review_display += "*"

    runtime_seconds = synset_execution_seconds.get(synset_id)
    runtime_display = f"{runtime_seconds:.2f}" if runtime_seconds is not None else "—"

    print(f"{i:<3} {synset_id:<18} {lemmas:<25} {pos:<5} {expanded_count:<10} {filtered_count:<10} {removed_count:<10} {confidence:<12} {def_review_display:<12} {runtime_display:<12}")

total_expanded = sum(len(r.get('payload', {}).get('expansion', {}).get('expanded_synonyms', []) or []) for r in batch_results)
total_filtered = sum(len(r.get('payload', {}).get('filtering', {}).get('filtered_synonyms', []) or []) for r in batch_results)
total_removed = sum(len(r.get('payload', {}).get('filtering', {}).get('removed', []) or []) for r in batch_results)

total_synsets = len(batch_results)

def pct(value: int) -> float:
    return 0.0 if total_synsets == 0 else (value / total_synsets * 100)

print("\n" + "=" * 112)
print("OVERALL STATISTICS")
print("=" * 112)
print(f"\n📈 Total candidates expanded: {total_expanded}")
print(f"✅ Total candidates filtered: {total_filtered}")
print(f"❌ Total candidates removed: {total_removed}")
if total_expanded:
    print(f"📉 Average removal rate: {(total_removed/total_expanded*100):.1f}%")
else:
    print("📉 Average removal rate: N/A")

high_conf = sum(1 for r in batch_results if r.get('payload', {}).get('filtering', {}).get('confidence') == 'high')
medium_conf = sum(1 for r in batch_results if r.get('payload', {}).get('filtering', {}).get('confidence') == 'medium')
low_conf = sum(1 for r in batch_results if r.get('payload', {}).get('filtering', {}).get('confidence') == 'low')

print(f"\n🎯 Overall confidence distribution:")
print(f"   🟢 High: {high_conf}/{total_synsets} synsets ({pct(high_conf):.0f}%)")
print(f"   🟡 Medium: {medium_conf}/{total_synsets} synsets ({pct(medium_conf):.0f}%)")
print(f"   🔴 Low: {low_conf}/{total_synsets} synsets ({pct(low_conf):.0f}%)")

print(f"\n🧹 Definition quality outcomes:")
print(f"   ok: {quality_counts['ok']}")
print(f"   needs_revision: {quality_counts['needs_revision']}")
print(f"   unknown/other: {quality_counts['unknown']}")
print(f"   Revisions applied (status '*'): {revision_count}")

if execution_summary:
    total_seconds = execution_summary.get("total_seconds")
    measured_sum = execution_summary.get("sum_synset_seconds")
    if total_seconds is not None:
        print(f"\n⏱️ Total runtime (wall-clock): {total_seconds:.2f} seconds")
    if measured_sum is not None and total_synsets:
        print(f"   Average per synset (measured): {measured_sum/total_synsets:.2f} seconds")

COMPARATIVE SUMMARY: All 5 Synsets

#   Synset ID          Lemmas                    POS   Expanded   Filtered   Removed    Confidence   Def.Review  
--------------------------------------------------------------------------------------------------------------
1   ENG30-03574555-n   institution               n     19         8          11         high         needs_revision*
2   ENG30-07810907-n   condiment                 n     16         8          8          high         needs_revision*
3   ENG30-01376245-v   scatter, sprinkle...      v     9          8          1          high         needs_revision*
4   ENG30-01382083-v   pick, pluck...            v     9          7          2          high         needs_revision*
5   ENG30-01393996-v   sweep                     v     9          4          5          high         needs_revision*

OVERALL STATISTICS

📈 Total candidates expanded: 62
✅ Total candidates filtered: 35
❌ Total candidates removed: 27
📉 Average removal rate: 43.5%

🎯 Overa

## 5️⃣ Comparison with Existing Serbian WordNet

Compare our pipeline output with human-created Serbian WordNet synsets.

In [13]:
# Compare with existing Serbian WordNet
if "batch_results" not in globals() or not batch_results:
    raise RuntimeError("Run the batch translation cell before running the comparison analysis.")

print("=" * 90)
print("PIPELINE vs EXISTING SERBIAN WORDNET")
print("=" * 90)

total_overlap = 0
total_existing = 0
total_our = 0

for idx, (name, synset, result, serbian_pair) in enumerate(
    zip(all_display_names, batch_synsets, batch_results, batch_pairs)
):
    payload = result.get('payload', {}) if isinstance(result, dict) else {}
    filtering = payload.get('filtering', {}) or {}
    definition_payload = payload.get('definition', {}) or {}
    definition_quality = payload.get('definition_quality', {}) or {}

    our_words = set(filtering.get('filtered_synonyms', []) or [])
    our_confidence = filtering.get('confidence', 'unknown')
    pre_review_definition = definition_payload.get('definition_translation', '')
    revised_definition = definition_quality.get('revised_definition')
    final_definition = result.get('definition_translation', pre_review_definition)

    their_words = set(serbian_pair.get('serbian_synonyms', []) or [])
    their_definition = serbian_pair.get('serbian_definition')

    overlap = our_words & their_words
    only_ours = our_words - their_words
    only_theirs = their_words - our_words

    print(f"\n{'='*90}")
    print(f"SYNSET {idx + 1}: {name.upper()}")
    print(f"ID: {synset['id']}")
    print(f"English Lemmas: {', '.join(synset['lemmas'])}")
    print(f"POS: {synset['pos']}")
    print(f"{'='*90}")

    print(f"\n📖 DEFINITION COMPARISON:")
    print(f"   🇬🇧 English: {synset['definition']}")
    print(f"   🇷🇸 Pipeline (final): {final_definition or '—'}")
    if revised_definition:
        print(f"   📝 Revised during quality review: {revised_definition}")
    if pre_review_definition and pre_review_definition != final_definition:
        print(f"   🔁 Pre-review version: {pre_review_definition}")
    print(f"   📚 Existing: {their_definition}")

    print(f"\n🆕 Pipeline ({len(our_words)} synonyms, {our_confidence} confidence):")
    print(f"   {', '.join(sorted(our_words)) if our_words else '—'}")

    print(f"\n📚 Existing WordNet ({len(their_words)} synonyms):")
    print(f"   {', '.join(sorted(their_words)) if their_words else '—'}")

    print(f"\n🔄 Synonym Analysis:")
    print(f"   ✅ Matches: {', '.join(sorted(overlap)) if overlap else 'None'}")
    print(f"   🆕 Only pipeline: {', '.join(sorted(only_ours)) if only_ours else 'None'}")
    print(f"   📚 Only existing: {', '.join(sorted(only_theirs)) if only_theirs else 'None'}")

    if len(their_words) > 0:
        match_rate = len(overlap) / len(their_words) * 100
        print(f"   📊 Match rate: {len(overlap)}/{len(their_words)} ({match_rate:.1f}%)")

    quality_status = (definition_quality.get('status') or 'unknown')
    issues = definition_quality.get('issues') or []
    notes = definition_quality.get('notes')

    print(f"\n🧹 Definition quality review: {quality_status}")
    if issues:
        print(f"   ⚠️ Issues flagged: {len(issues)}")
        for issue in issues:
            print(f"     - {issue.get('type', 'unknown')}: {issue.get('message', '')}")
    if notes:
        print(f"   🗒️ Reviewer notes: {notes}")

    total_overlap += len(overlap)
    total_existing += len(their_words)
    total_our += len(our_words)

print(f"\n{'='*90}")
print("OVERALL COMPARISON STATISTICS")
print(f"{'='*90}")
print(f"\n📊 Total synonyms:")
print(f"   Pipeline: {total_our}")
print(f"   Existing: {total_existing}")
print(f"   Matches: {total_overlap}")
if total_existing:
    print(f"   Overall match rate: {total_overlap}/{total_existing} ({total_overlap/total_existing*100:.1f}%)")
else:
    print("   Overall match rate: N/A")

print(f"\n💡 NOTE: Both definitions and synonyms should be compared.")
print(f"   Definitions show the semantic boundaries of each synset.")
print(f"   Matching definitions = same concept, even with different synonyms.")

PIPELINE vs EXISTING SERBIAN WORDNET

SYNSET 1: INSTITUTION
ID: ENG30-03574555-n
English Lemmas: institution
POS: n

📖 DEFINITION COMPARISON:
   🇬🇧 English: an establishment consisting of a building or complex of buildings where an organization for the promotion of some cause is situated
   🇷🇸 Pipeline (final): objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   📝 Revised during quality review: objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   🔁 Pre-review version: zgrada ili kompleks zgrada u kome se nalazi organizacija za promovisanje nekog cilja
   📚 Existing: zgrada u kojoj se nalazi organizaciona jedinica neke grane javnog poslovanja

🆕 Pipeline (8 synonyms, high confidence):
   administrativna zgrada, administrativni objekat, centar, centrala, filijala, glavna zgrada, radna zgrada, sedište

📚 Existing WordNet (1 synonyms):
   ustanova

🔄 Synonym Analysis:
   ✅ Matches: None
   🆕 Only pip

### Definition Quality Analysis

Definitions are the **foundation** of WordNet structure. They:
- Define semantic boundaries of each synset
- Determine which synonyms belong together
- Enable cross-lingual alignment
- Show the precise concept being represented

Let's analyze how well our pipeline's definitions match existing Serbian glosses.

In [14]:
# Detailed definition comparison
if "batch_results" not in globals() or not batch_results:
    raise RuntimeError("Run the batch translation cell before the detailed definition analysis.")

print("=" * 90)
print("DEFINITION QUALITY ANALYSIS")
print("=" * 90)
print("\nComparing pipeline-generated definitions with existing Serbian glosses.\n")

for idx, (name, synset, result, serbian_pair) in enumerate(
    zip(all_display_names, batch_synsets, batch_results, batch_pairs), start=1
):
    payload = result.get('payload', {}) if isinstance(result, dict) else {}
    definition_payload = payload.get('definition', {}) or {}
    definition_quality = payload.get('definition_quality', {}) or {}

    english_def = synset['definition']
    existing_def = serbian_pair.get('serbian_definition')
    pre_review_def = definition_payload.get('definition_translation', '')
    final_def = result.get('definition_translation', pre_review_def)
    revised_def = definition_quality.get('revised_definition')
    quality_status = (definition_quality.get('status') or 'unknown')
    issues = definition_quality.get('issues') or []
    notes = definition_quality.get('notes')

    print(f"\n{'─'*90}")
    print(f"📌 SYNSET {idx}: {name.upper()}")
    print(f"   ID: {synset['id']}")
    print(f"   English Lemmas: {', '.join(synset['lemmas'])}")
    print(f"   POS: {synset['pos']}")
    print(f"{'─'*90}")

    print(f"\n🇬🇧 English:")
    print(f"   {english_def}")

    print(f"\n🤖 Pipeline definition (final):")
    print(f"   {final_def or '—'}")
    if revised_def:
        print(f"   📝 Revised during quality review: {revised_def}")
    if pre_review_def and pre_review_def != final_def:
        print(f"   🔁 Pre-review version: {pre_review_def}")

    print(f"\n📚 Existing gloss:")
    print(f"   {existing_def}")

    pipeline_words = set((final_def or '').lower().split())
    existing_words = set((existing_def or '').lower().split())
    common_words = pipeline_words & existing_words

    if existing_words:
        overlap_pct = len(common_words) / len(existing_words) * 100
        print(f"\n📊 Word overlap (final vs. existing): {len(common_words)}/{len(existing_words)} words ({overlap_pct:.1f}%)")
        if common_words:
            print(f"   Common: {', '.join(sorted(common_words))}")

    print(f"\n🧹 Definition quality review: {quality_status}")
    if issues:
        print(f"   ⚠️ Issues flagged: {len(issues)}")
        for issue in issues:
            print(f"     - {issue.get('type', 'unknown')}: {issue.get('message', '')}")
    if notes:
        print(f"   🗒️ Notes: {notes}")

print(f"\n{'='*90}")
print("DEFINITION ANALYSIS SUMMARY")
print(f"{'='*90}")
print("\n💡 Key Insight: Definition alignment is crucial for WordNet quality.")
print("   - If definitions match → synonyms should align conceptually")
print("   - If definitions differ → may indicate different senses or translation issues")
print("   - Quality review catches stylistic or grammatical issues before finalizing")

DEFINITION QUALITY ANALYSIS

Comparing pipeline-generated definitions with existing Serbian glosses.


──────────────────────────────────────────────────────────────────────────────────────────
📌 SYNSET 1: INSTITUTION
   ID: ENG30-03574555-n
   English Lemmas: institution
   POS: n
──────────────────────────────────────────────────────────────────────────────────────────

🇬🇧 English:
   an establishment consisting of a building or complex of buildings where an organization for the promotion of some cause is situated

🤖 Pipeline definition (final):
   objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   📝 Revised during quality review: objekat – zgrada ili skup zgrada – u kojem se nalazi organizacija koja promoviše određeni cilj
   🔁 Pre-review version: zgrada ili kompleks zgrada u kome se nalazi organizacija za promovisanje nekog cilja

📚 Existing gloss:
   zgrada u kojoj se nalazi organizaciona jedinica neke grane javnog poslovanja

📊 Word 

## 6️⃣ Key Findings

### Definition Translation Quality
- 🎯 **Most important metric**: Definitions determine synset boundaries
- ✅ Pipeline translates definitions with cultural adaptation
- ✅ Definitions should be compared with existing glosses
- ✅ Matching definitions = same concept, validates synonym choices
- ⚠️ Definition differences may indicate sense drift or translation issues

### Iterative Expansion
- ✅ Runs expansion multiple times until no new synonyms appear
- ✅ Typical convergence: 2-3 iterations
- ✅ Ensures comprehensive coverage despite LLM variability

### Improved Filtering (Definition-Anchored)
- ✅ Filters synonyms strictly against the translated definition
- ✅ Rejects candidates matching different senses of polysemous words
- ✅ Removes modifiers/particles that shift meaning
- ✅ Keeps only canonical lemmas expressing the exact concept

### Definition Quality Review
- ✅ Automated grammar/style sweep catches lingering issues
- ✅ Can replace the definition when `needs_revision` is returned
- ✅ Logs issues and reviewer notes for lexicographers
- ✅ Final curator summary shows review status for every synset

### Per-Word Confidence
- ✅ Individual quality scores for each synonym
- ✅ Enables threshold-based filtering
- ✅ High confidence rate: ~80% of synsets

### Comparison with Existing WordNet
- The goal is not 100% match, but **complementary** suggestions
- Pipeline may find valid synonyms humans didn't include
- Humans may include domain-specific terms pipeline misses
- **Both definitions AND synonyms must be evaluated together**
- Both approaches have value for lexicographers

### 📁 Logging Artifacts
Persist the full prompt/response history from every stage so you can review it later or attach it to QA reports. The JSON file includes both the single-synset demo and the batch run.

In [ ]:
from datetime import datetime, timezone
import re

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

has_single = "result_0" in globals()
has_batch = "batch_results" in globals() and bool(batch_results)

if not (has_single or has_batch):
    print("⚠️ No translation outputs are loaded yet. Run the single-synset and/or batch translation cells before exporting logs.")
else:
    timestamp = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
    model_safe = re.sub(r"[^A-Za-z0-9_-]+", "-", model)
    log_path = OUTPUT_DIR / f"pipeline_logs_{model_safe}_{timestamp}.json"

    execution_summary = globals().get("batch_execution_summary")
    synset_execution_seconds = globals().get("synset_execution_seconds", {})
    single_synset_id = synset_0["id"] if "synset_0" in globals() else None

    log_bundle = {
        "metadata": {
            "model": model,
            "timestamp_utc": timestamp,
            "source_lang": pipeline.source_lang,
            "target_lang": pipeline.target_lang,
            "max_expansion_iterations": pipeline.max_expansion_iterations,
        },
        "runs": [],
    }

    if execution_summary:
        if execution_summary.get("total_seconds") is not None:
            log_bundle["metadata"]["batch_total_seconds"] = execution_summary["total_seconds"]
        if execution_summary.get("sum_synset_seconds") is not None:
            log_bundle["metadata"]["batch_sum_synset_seconds"] = execution_summary["sum_synset_seconds"]
    if single_synset_id and synset_execution_seconds.get(single_synset_id) is not None:
        log_bundle["metadata"]["single_demo_seconds"] = synset_execution_seconds[single_synset_id]

    def _add_run(label: str, synset_id: str, result_dict: dict) -> None:
        payload = result_dict.get("payload", {}) if isinstance(result_dict, dict) else {}
        run_entry = {
            "label": label,
            "synset_id": synset_id,
            "calls": payload.get("calls", {}),
            "logs": payload.get("logs", {}),
        }
        runtime_value = synset_execution_seconds.get(synset_id)
        if runtime_value is not None:
            run_entry["execution_seconds"] = runtime_value
        log_bundle["runs"].append(run_entry)

    if has_single:
        _add_run("single_demo", synset_0["id"], result_0)

    if has_batch:
        for synset, result, name in zip(batch_synsets, batch_results, all_display_names):
            _add_run(f"batch_{synset['id']}", synset["id"], result)

    if not log_bundle["runs"]:
        print("⚠️ No runs available to export yet. Complete at least one translation stage before saving logs.")
    else:
        with log_path.open("w", encoding="utf-8") as f:
            json.dump(log_bundle, f, ensure_ascii=False, indent=2)

        print(f"📄 Saved detailed stage logs to: {log_path.resolve()}")
        print("   • File name includes model and UTC timestamp for easy comparison across runs.")
        print("   • Contains prompts, raw responses, parsed payloads, condensed summaries, and execution timing for every synset processed.")

📄 Saved detailed stage logs to: E:\Github\wordnet_autotranslate\notebooks\output\pipeline_logs_gpt-oss-120b_20251020T165650Z.json
   • File name includes model and UTC timestamp for easy comparison across runs.
   • Contains prompts, raw responses, parsed payloads, and condensed summaries for every synset processed.


### 📦 Structured Result Export
Capture a full snapshot of each translated synset—including final definitions, synonym confidence, removed candidates, and review notes—so you can compare runs across models and timestamps.

In [ ]:
from datetime import datetime, timezone
import re

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

has_single = "result_0" in globals()
has_batch = "batch_results" in globals() and bool(batch_results)

if not (has_single or has_batch):
    print("⚠️ No translation outputs are loaded yet. Run the single-synset and/or batch translation cells before exporting results.")
else:
    timestamp_results = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
    model_safe = re.sub(r"[^A-Za-z0-9_-]+", "-", model)
    results_path = OUTPUT_DIR / f"pipeline_results_{model_safe}_{timestamp_results}.json"

    execution_summary = globals().get("batch_execution_summary")
    synset_execution_seconds = globals().get("synset_execution_seconds", {})
    single_synset_id = synset_0["id"] if "synset_0" in globals() else None

    def _collect_entry(synset_data: dict, result_dict: dict, display_name: str | None = None) -> dict:
        payload = result_dict.get("payload", {}) if isinstance(result_dict, dict) else {}
        sense_payload = payload.get("sense", {}) or {}
        definition_payload = payload.get("definition", {}) or {}
        definition_quality_payload = payload.get("definition_quality", {}) or {}
        initial_payload = payload.get("initial_translation", {}) or {}
        expansion_payload = payload.get("expansion", {}) or {}
        filtering_payload = payload.get("filtering", {}) or {}

        confidence_by_word = filtering_payload.get("confidence_by_word", {}) or {}
        filtered_synonyms = filtering_payload.get("filtered_synonyms", []) or []
        synonyms_with_conf = [
            {
                "word": str(word),
                "confidence": confidence_by_word.get(word)
            }
            for word in filtered_synonyms
        ]

        removed_candidates = filtering_payload.get("removed", []) or []
        if isinstance(removed_candidates, dict):
            removed_candidates = [removed_candidates]

        runtime_value = synset_execution_seconds.get(synset_data.get("id"))

        entry = {
            "synset": {
                "id": synset_data.get("id"),
                "lemmas": synset_data.get("lemmas", []),
                "definition": synset_data.get("definition"),
                "examples": synset_data.get("examples", []),
                "pos": synset_data.get("pos"),
                "display_name": display_name or synset_data.get("id"),
            },
            "final_output": {
                "representative_literal": result_dict.get("translation"),
                "definition": result_dict.get("definition_translation"),
                "synonyms": synonyms_with_conf,
                "overall_confidence": filtering_payload.get("confidence"),
                "removed_candidates": removed_candidates,
                "notes": result_dict.get("notes") or definition_payload.get("notes") or definition_quality_payload.get("notes"),
            },
            "definition_quality": {
                "status": definition_quality_payload.get("status"),
                "revised_definition": definition_quality_payload.get("revised_definition"),
                "issues": definition_quality_payload.get("issues", []),
                "notes": definition_quality_payload.get("notes"),
            },
            "stages": {
                "sense_analysis": sense_payload,
                "definition_translation": definition_payload,
                "initial_translation": initial_payload,
                "expansion": expansion_payload,
                "filtering": filtering_payload,
            },
            "examples": result_dict.get("examples", []),
            "curator_summary": result_dict.get("curator_summary"),
            "raw_response": result_dict.get("raw_response"),
        }

        if runtime_value is not None:
            entry["execution"] = {
                "seconds": runtime_value
            }

        related_log = globals().get("log_path")
        if related_log is not None:
            entry["related_log_file"] = str(Path(related_log).resolve())

        return entry

    result_entries = []

    if has_batch:
        for syn, res, name in zip(batch_synsets, batch_results, all_display_names):
            result_entries.append(_collect_entry(syn, res, name))
    else:
        if has_single:
            result_entries.append(_collect_entry(synset_0, result_0, synset_0.get("id")))

    if not result_entries:
        print("⚠️ No completed translations found to export. Finish running the pipeline cells first.")
    else:
        results_bundle = {
            "metadata": {
                "model": model,
                "timestamp_utc": timestamp_results,
                "source_lang": pipeline.source_lang,
                "target_lang": pipeline.target_lang,
                "max_expansion_iterations": pipeline.max_expansion_iterations,
            },
            "entries": result_entries,
        }

        if execution_summary:
            if execution_summary.get("total_seconds") is not None:
                results_bundle["metadata"]["batch_total_seconds"] = execution_summary["total_seconds"]
            if execution_summary.get("sum_synset_seconds") is not None:
                results_bundle["metadata"]["batch_sum_synset_seconds"] = execution_summary["sum_synset_seconds"]
        if not has_batch and single_synset_id and synset_execution_seconds.get(single_synset_id) is not None:
            results_bundle["metadata"]["single_demo_seconds"] = synset_execution_seconds[single_synset_id]

        with results_path.open("w", encoding="utf-8") as f:
            json.dump(results_bundle, f, ensure_ascii=False, indent=2)

        print(f"💾 Saved structured pipeline results to: {results_path.resolve()}")
        print("   • Includes final definitions, synonym confidence, removed candidates, stage payloads, and execution timings for each synset.")
        print("   • Use the model/timestamp in the filename to compare across different runs.")

💾 Saved structured pipeline results to: E:\Github\wordnet_autotranslate\notebooks\output\pipeline_results_gpt-oss-120b_20251020T165651Z.json
   • Includes final definitions, synonym confidence, removed candidates, and stage payloads for each synset.
   • Use the model/timestamp in the filename to compare across different runs.
